In [2]:
import pandas as pd

In [3]:
# read the purchase_data.json
jsonPath = "purchase_data.json"
playerJson = pd.read_json(jsonPath)
playerJson.head()

,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59


Total Number of Players

In [11]:
#Total Number of Players
dedupePlayer = pd.DataFrame(playerJson.groupby('SN').count())
totalNumPlayers =len(dedupePlayer)
totalNumPlayers_df = pd.DataFrame({"Total Players":totalNumPlayers},index = {0})
totalNumPlayers_df

,Total Players
0,573


Purchasing Analysis (Total)

In [16]:
# Number of Unique Items
# Average Purchase Price
# Total Number of Purchases
# Total Revenue

#Number of Unique Items
dedupeItem = pd.DataFrame(playerJson.groupby('Item ID').count())
totalUniqueItems = len(dedupeItem)

#Average Purchase Price
purchasePriceSeries = playerJson.loc[:,'Price']
avgPurchasePrice = round(purchasePriceSeries.mean(),2)

#Total Number of Purchases
numberPurchases = len(playerJson)

#Total Revenue
totalRevenue = round(playerJson.loc[:,'Price'].sum(),2)

#Purchasing Analysis Summary
purchasingAnalysisSummary_df = pd.DataFrame({"Num of Unique Items":totalUniqueItems,\
                                        "Avg Purchase Price":avgPurchasePrice,\
                                        "Total Purchases":numberPurchases,\
                                        "Total Revenue":totalRevenue},index ={0})
purchasingAnalysisSummary_df = purchasingAnalysisSummary_df[["Num of Unique Items",\
                                        "Avg Purchase Price",\
                                        "Total Purchases",\
                                        "Total Revenue"]]
purchasingAnalysisSummary_df

,Num of Unique Items,Avg Purchase Price,Total Purchases,Total Revenue
0,183,2.93,780,2286.33


Gender Demographics

In [30]:
#Percentage and Count of Gender

#Dedupe the players
dedupeGender = playerJson.groupby(['SN','Gender']).count()

#Count by Gender
dedupeGender_df = pd.DataFrame(dedupeGender.groupby('Gender').count())
dedupeGenderCount = dedupeGender_df['Age'].rename("Count")

#Percentage by Gender
dedupeGenderPercentage = round(dedupeGenderCount/totalNumPlayers * 100,2).rename("Percentage")

#Percentage and Count of Gender Table
genderCountSummary_df = pd.DataFrame({"Percentage of Players":dedupeGenderPercentage,\
                                        "Total Count":dedupeGenderCount})
genderCountSummary_df = genderCountSummary_df[["Percentage of Players",\
                                        "Total Count"]]
genderCountSummary_df


,Percentage of Players,Total Count
Gender,,
Female,17.45,100
Male,81.15,465
Other / Non-Disclosed,1.40,8


Purchasing Analysis (Gender)

In [45]:
#Create a purchase table with Gender as groupby
genderGroupby = playerJson.groupby(['Gender'])

#Purchase Count
genderPurchaseCount = genderGroupby["Item ID"].count()

#Average Purchase Price
genderAvgPurchase = round(genderGroupby["Price"].mean(),2)

#Total Purchase Value
genderTotalPurchase = round(genderGroupby["Price"].sum(),2)

#Normalized Totals
genderNorPurchase = round(genderTotalPurchase/dedupeGenderCount,2)

#Gender Purchasing Summary
genderPurchaseSummary_df = pd.DataFrame({"Purchase Count":genderPurchaseCount,\
                                        "Avg Purchase Price":genderAvgPurchase,\
                                        "Total Purchase Value":genderTotalPurchase,\
                                        "Normalized Totals":genderNorPurchase})
genderPurchaseSummary_df = genderPurchaseSummary_df[["Purchase Count",\
                                                     "Avg Purchase Price",\
                                                     "Total Purchase Value",\
                                                     "Normalized Totals"]]
genderPurchaseSummary_df


,Purchase Count,Avg Purchase Price,Total Purchase Value,Normalized Totals
Gender,,,,
Female,136,2.82,382.91,3.83
Male,633,2.95,1867.68,4.02
Other / Non-Disclosed,11,3.25,35.74,4.47


In [48]:
#Age Demographics
#The below each broken into bins of 4 years (i.e. &lt;10, 10-14, 15-19, etc.) 

#Add an extra column to raw table with the bins for age
bins = [0, 10, 14, 19, 24, 29, 34, 39,playerJson["Age"].max()]
group_names = ["<10", "10-14", "15-19", "20-24","25-29","30-34","35-39","40+"]
playerJson["Age Bin"] = pd.cut( playerJson["Age"], bins, labels=group_names)
ageBinGroupby = playerJson.groupby(['Age Bin'])

#Purchase Count
agePurchaseCount = ageBinGroupby["Item ID"].count()

#Average Purchase Price
ageAvgPurchase = round(ageBinGroupby["Price"].mean(),2)

#Total Purchase Value
ageTotalPurchase = round(ageBinGroupby["Price"].sum(),2)

#Normalized Totals
#dedupe age using the player dedupe table
groupPlayerAge = playerJson.groupby(["SN","Age"]).count()
groupPlayerAge = groupPlayerAge.reset_index(level='Age', col_level=1)
groupPlayerAge["Age Bin"] =  pd.cut( groupPlayerAge["Age"], bins, labels=group_names)
groupAgeBin = groupPlayerAge.groupby(['Age Bin']).count()

ageNorPurchase = round(ageTotalPurchase/groupAgeBin['Age'],2)

#Age Demographic Summary
ageDemographicSummary_df = pd.DataFrame({"Purchase Count":agePurchaseCount,\
                                        "Avg Purchase Price":ageAvgPurchase,\
                                        "Total Purchase Value":ageTotalPurchase,\
                                        "Normalized Totals":ageNorPurchase})
ageDemographicSummary_df = ageDemographicSummary_df[["Purchase Count",\
                                                     "Avg Purchase Price",\
                                                     "Total Purchase Value",\
                                                     "Normalized Totals"]]
ageDemographicSummary_df


,Purchase Count,Avg Purchase Price,Total Purchase Value,Normalized Totals
Age Bin,,,,
<10,32,3.02,96.62,4.39
10-14,31,2.70,83.79,4.19
15-19,133,2.91,386.42,3.86
20-24,336,2.91,978.77,3.78
25-29,125,2.96,370.33,4.26
30-34,64,3.08,197.25,4.20
35-39,42,2.84,119.40,4.42
40+,17,3.16,53.75,4.89


Top Spenders

In [74]:
#Identify the the top 5 spenders in the game by total purchase value, then list (in a table):
# SN, Purchase Count, Average Purchase Value, Total Purchase Value

#Create table that groupby SN
snGroupby = playerJson.groupby(['SN'])
snPurchaseCount = round(snGroupby["Item ID"].count(),2)
snAvgPurchase = round(snGroupby["Price"].mean(),2)
snTotalPurchase = round(snGroupby["Price"].sum(),2)
snSummary_df = pd.DataFrame({"Purchase Count":snPurchaseCount,\
                             "Average Purchase Value":snAvgPurchase,\
                             "Total Purchase Value":snTotalPurchase})

#Sort by total purchase value then take the top 5
snTop5Summary_df = snSummary_df.nlargest(5,'Total Purchase Value', keep='first')
snTop5Summary_df = snTop5Summary_df[["Purchase Count",\
                                    "Average Purchase Value",\
                                    "Total Purchase Value"]]
snTop5Summary_df

,Purchase Count,Average Purchase Value,Total Purchase Value
SN,,,
Undirrala66,5,3.41,17.06
Saedue76,4,3.39,13.56
Mindimnya67,4,3.18,12.74
Haellysu29,3,4.24,12.73
Eoda93,3,3.86,11.58


Most Popular Items

In [78]:
#Identify the 5 most popular items by purchase count, then list (in a table):
# Item ID, Item Name, Purchase Count, Item Price, Total Purchase Value

#Create table that groupby Item ID
itemGroupby = playerJson.groupby(['Item ID','Item Name', 'Price'])
itemPurchaseCount = round(itemGroupby["Price"].count(),2)
itemAvgPurchase = round(itemGroupby["Price"].mean(),2)
itemTotalPurchase = round(itemGroupby["Price"].sum(),2)
itemSummary_df = pd.DataFrame({"Purchase Count":itemPurchaseCount,\
                               "Total Purchase Value":itemTotalPurchase})

#Sort by total purchase count then take the top 5
itemTop5CountSummary_df = itemSummary_df.nlargest(5,'Purchase Count', keep='first')
itemTop5CountSummary_df = itemTop5CountSummary_df[["Purchase Count",\
                                    "Total Purchase Value"]]
itemTop5CountSummary_df

,,,Purchase Count,Total Purchase Value
Item ID,Item Name,Price,,
39,"Betrayal, Whisper of Grieving Widows",2.35,11,25.85
84,Arcane Gem,2.23,11,24.53
13,Serenity,1.49,9,13.41
31,Trickster,2.07,9,18.63
34,Retribution Axe,4.14,9,37.26


Most Profitable Items

In [79]:
#Identify the 5 most profitable items by total purchase value, then list (in a table):
# Item ID, Item Name, Purchase Count, Item Price, Total Purchase Value
itemTop5ValueSummary_df = itemSummary_df.nlargest(5,'Total Purchase Value', keep='first')
itemTop5ValueSummary_df = itemTop5ValueSummary_df[["Purchase Count",\
                                    "Total Purchase Value"]]
itemTop5ValueSummary_df

,,,Purchase Count,Total Purchase Value
Item ID,Item Name,Price,,
34,Retribution Axe,4.14,9,37.26
115,Spectral Diamond Doomblade,4.25,7,29.75
32,Orenmir,4.95,6,29.70
103,Singed Scalpel,4.87,6,29.22
107,"Splitter, Foe Of Subtlety",3.61,8,28.88


Observations of three trends:
1. Males buy more products, more expensive items, and more frequently than Females.
2. 20-24 age group buy the most products but 40+ buy more expensive products and more frequently out of any age group.
3. The frequency of purchases per user is low since the top user in purchase value only bought 5 times.